In [2]:
import pygad
import numpy

# Params
print("Ingrese los valores->")
# Population
initial_population = 50 #int(input("Tamaño población: ")) #50

# Mutations
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 5 #int(input("Mutación: ")) #5%

# Generations
num_generations = 100 #int(input("Num Generaciones: ")) #100
num_parents_mating = 7 # Number of solutions to be selected as parents in the mating pool.

# Solutions
sol_per_pop = 100 # Number of solutions in the population.
num_genes = 10

# Parents
parent_selection_type = "tournament"  # Type of parent selection.
keep_parents = 7 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.
crossover_type = "single_point" # Type of the crossover operator.

# Archive
url_archive = input("Ubicación web del archivo: ") #url

print("Listo para iniciar->")

Ingrese los valores->
Listo para iniciar->


In [3]:
n = 10 # Tamaño del cuadro
M = 2 # Cantidad de mesas (2)
P = 2 # Cantidad de puertas (3)
O = 25 # Espacio no ocupable (4)

# Archive
plano = numpy.array([
    [0,0,0,0,0,0,0,3,0,0],
    [2,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,4,4,4],
    [0,0,0,0,0,0,0,4,4,4],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,4,4,4,4],
    [0,0,0,0,0,2,4,4,4,4],
    [0,0,0,0,0,0,4,4,4,4],
    [3,0,0,0,0,0,4,4,4,4]
])
print(plano)
# Reading archive


[[0 0 0 0 0 0 0 3 0 0]
 [2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 4 4]
 [0 0 0 0 0 0 0 4 4 4]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 4 4 4 4]
 [0 0 0 0 0 2 4 4 4 4]
 [0 0 0 0 0 0 4 4 4 4]
 [3 0 0 0 0 0 4 4 4 4]]


In [ ]:



def fitness_func(solution, solution_idx):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.

    return fitness

fitness_function = fitness_func

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed),
    ",   Fitness Best   = {fitness:.3f}".format(fitness=ga_instance.best_solution()[1]),
    ",   Change     = {change:.3f}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       fitness_func=fitness_function,
                       parent_selection_type=parent_selection_type,
                       gene_space=[0, 1],
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# Saving the GA instance.
filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
ga_instance.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_result()